In [1]:
import os 
import getopt
import sys

import random
import re

import numpy as np
import h5py
import pickle
import random
import copy
import pandas as pd
import math 

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Masking, Lambda, concatenate, Bidirectional, Dense, Dropout, Flatten, Conv1D,BatchNormalization,  MaxPooling1D, Bidirectional, GRU, TimeDistributed
import tensorflow as tf
from tensorflow import keras



np.random.seed(1337) # for reproducibility
vocab = ["A", "G", "C", "T"]
indices = tf.range(len(vocab), dtype = tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab,indices)
table = tf.lookup.StaticVocabularyTable(table_init, 1)
record_defaults = [
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),  
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),  
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
]

# Nadav dataset

def data_reader(file, batch_size=100, n_parse_threads=4):
    """Method for reading the data in an optimized way, can be used inside model.fit()
    
    Args:
        file (_type_): path to csv file
        batch_size (int, optional): _description_. Defaults to 100.
        n_parse_threads (int, optional): _description_. Defaults to 4.

    Returns:
        dataset.batch: batch dataset object 
    """
    dataset = tf.data.TextLineDataset(file).skip(1)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)

def preprocess(record):
    """Preprocessing method of a dataset object, one-hot-encodes the data

    Args:
        record (_type_): _description_

    Returns:
        X (2D np.array): one-hot-encoded input sequence
        Y (1D np.array): MPRA measurements for each cell state
    """
    # Get fields from the data
    fields = tf.io.decode_csv(record, record_defaults=record_defaults)
    
    # One-hot-encode data
    chars = tf.strings.bytes_split(fields[0])
    chars_indices = table.lookup(chars)
    
    # Create a mask for out-of-vocabulary characters
    oov_mask = tf.equal(chars_indices, len(vocab))
    
    # Explicitly cast the value to int64
    chars_indices = tf.where(oov_mask, tf.constant(len(vocab), dtype=tf.int64), chars_indices)
    
    X = tf.one_hot(chars_indices, depth = len(vocab))
    
    # Combine y for each cell type into one vector 
    Y = tf.stack(fields[2:])
    
    # Replace missing values with -1
    Y= tf.where(tf.equal(Y,  "nan"), ["-1"], Y)
    Y= tf.where(tf.equal(Y,  "NA"), ["-1"], Y)
    Y = tf.strings.to_number(Y, tf.float32)
    return X,Y

# Get first item of the dataset to get the shape of the input data
for element in data_reader("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/no_upper_LibA_wide_pivot_all_states.csv"):
    input_shape = element[0].shape
    output_shape = element[1].shape
    
print(output_shape)
print(input_shape)

2023-11-28 13:55:01.519348: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-28 13:55:01.652907: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 13:55:01.652949: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 13:55:01.653536: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 13:55:01.701415: I tensorflow/core/platform/cpu_feature_g

(92, 7)
(92, 262, 4)


In [2]:
# Define the masking layer with mask_value=0
masking_layer = tf.keras.layers.Masking(mask_value=0.0)

# Example input tensor with shape (batch_size, sequence_length, vocab_size)
input_tensor = tf.constant([
    [[0, 0, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 0, 1, 0]],
    [[0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0]]
], dtype=tf.float32)

# Apply the masking layer to the input tensor
masked_tensor = masking_layer(input_tensor)

print(masked_tensor)
masking_layer.compute_mask(input_tensor)

tf.Tensor(
[[[0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]]

 [[0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0.]]], shape=(2, 3, 5), dtype=float32)


<tf.Tensor: shape=(2, 3), dtype=bool, numpy=
array([[False,  True,  True],
       [ True,  True, False]])>

In [3]:
# CROSS VALIDATION (10 fold)
from sklearn.model_selection import train_test_split, KFold

whole_data = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/2bp_upper_LibA_wide_pivot_all_states.csv"
out_data = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/"
# Split the data in two partitions
whole_data = pd.read_csv(whole_data)
k = 10
kf = KFold(n_splits = k, shuffle = True, random_state = 2008)

o=1
# For each fold we split again to get the third partition
for i in kf.split(whole_data):
    
    # Extract data from fold
    train = whole_data.iloc[i[0]]
    test =  whole_data.iloc[i[1]]
    
    # Split into train, validation and test
    valid, validation = train_test_split(train, test_size=0.111, random_state=42)
    
    # Randomize motifs
    #train = randomize_motifs_and_augment(train, num_augmentations=100)
    #train["rnd_seq"] = train['rnd_seq'].str.lower() 
    #test = randomize_motifs_and_augment(test, num_augmentations=100)
    #test["rnd_seq"] = test['rnd_seq'].str.lower() 
    #validation = randomize_motifs_and_augment(validation, num_augmentations=100)
    #validation["rnd_seq"] = validation['rnd_seq'].str.lower() 
    
    train.astype(str).to_csv(out_data+"all_states_2bp_added_CV"+str(o)+"_LibA_wide_pivot_state3_train.csv", index=False)
    test.astype(str).to_csv(out_data+"all_states_2bp_added_CV"+str(o)+"_LibA_wide_pivot_state3_test.csv", index=False)
    validation.astype(str).to_csv(out_data+"all_states_2bp_added_CV"+str(o)+"_LibA_wide_pivot_state3_validation.csv", index=False)
    o+=1
    

In [4]:
# Here we initialize the df where each fold test prediction will be appended to
# the list containing the correlations of each fold is also initialized
corr_list = []

# We define a custom normalization layer to then compile on the model
class CustomNormalization(Layer):
    """Custom normalization layer that normalizes the output of the neural network"""
    def __init__(self, **kwargs):
        super(CustomNormalization, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Add trainable variables for mean and standard deviation
        self.mean = self.add_weight("mean", shape=(1,), initializer="zeros", trainable=True)
        self.stddev = self.add_weight("stddev", shape=(1,), initializer="ones", trainable=True)
        super(CustomNormalization, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        # Normalize the inputs using the learned mean and standard deviation
        return (inputs - self.mean) / (self.stddev + 1e-8)

# We define the method to compute the pearson correlation between prediction and ground truth in the multi_head case
def pearson_correlation_multi_head(predictions, ground_truth, mask_value=-1):
    """Computes Pearson Correlation between predictions and ground truth for each column
    Args:
        predictions (np.array): 2D array of prediction values (N, 7)
        ground_truth (np.array): 2D array of ground truth values (N, 7)
        mask_value (float): Value in ground truth to be ignored in correlation computation

    Returns:
        correlations (np.array): 1D array of Pearson correlations for each column
    """
    # Ensure predictions and ground_truth have the same shape
    assert predictions.shape == ground_truth.shape, "Input shapes do not match"

    n_columns = predictions.shape[1]
    correlations = np.zeros(n_columns)

    for col in range(n_columns):
        x = predictions[:, col]
        y = ground_truth[:, col]

        # Exclude values in ground truth equal to mask_value
        valid_indices = (y != mask_value)
        x = x[valid_indices]
        y = y[valid_indices]

        if len(x) == 0 or len(y) == 0:
            # If no valid values, set correlation to NaN
            correlations[col] = np.nan
        else:
            # Calculate mean of x and y
            mean_x = np.mean(x)
            mean_y = np.mean(y)

            # Calculate the numerator and denominators of the correlation coefficient
            numerator = np.sum((x - mean_x) * (y - mean_y))
            denominator_x = np.sqrt(np.sum((x - mean_x) ** 2))
            denominator_y = np.sqrt(np.sum((y - mean_y) ** 2))

            # Calculate the correlation coefficient
            correlation = numerator / (denominator_x * denominator_y)
            correlations[col] = correlation

    return correlations



import matplotlib.pyplot as plt
%matplotlib inline
# Define plotting function of loss
def create_plots(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.clf()
    

In [5]:
# Define a custom loss function
class MaskedMSE(tf.keras.losses.Loss):
    """Computes the MSE loss and prevents missing values backpropagation (previously replaced by -1.0)

    Args:
        tf (_type_): _description_
    """
    def __init__(self, mask_value=-1, **kwargs):
        super(MaskedMSE, self).__init__(**kwargs)
        self.mask_value = mask_value

    def call(self, y_true, y_pred):
        # Create a mask for valid elements (not equal to the specified mask_value)
        mask = tf.math.not_equal(y_true, self.mask_value)

        # Compute MSE loss only for valid elements
        loss = tf.where(mask, tf.square(y_true - y_pred), 0.0)

        # Calculate the mean loss
        mean_loss = tf.reduce_sum(loss) / tf.reduce_sum(tf.cast(mask, dtype=tf.float32))

        return mean_loss

In [6]:
from tensorflow.keras.optimizers import Adam

corr_coefficients = {}

df_test_10folds  = pd.DataFrame(columns=['State_3E',
                                         "seq",
                                         "avg_prediction_State_1M",
                                         "avg_prediction_State_2D",
                                         "avg_prediction_State_3E",
                                         "avg_prediction_State_4M",
                                         "avg_prediction_State_5M",
                                         "avg_prediction_State_6N",
                                         "avg_prediction_State_7M"])
# We iterate through each of the train folds to train, test and validate the model

for i in range(1,11):
    #Define inputs
    input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_2bp_added_CV"+str(i)+"_LibA_wide_pivot_state3_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_2bp_added_CV"+str(i)+"_LibA_wide_pivot_state3_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_2bp_added_CV"+str(i)+"_LibA_wide_pivot_state3_test.csv"
    
    # Read test data to then predict
    df_test = pd.read_csv(input_path_test)
    df_test["fold"] = str(i)
    
    predictions_sum = np.zeros((df_test.shape[0], 7))
    
    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape
    
    correlation_ensemble = []
    # Initialize vector containing predictions
    
    for i in range(1,11):
        
        tf.random.set_seed(i)
        
        # Define and compile model
        inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
        inputs = Masking()(inputs)
        layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
        layer = Dropout(0.5)(layer)
        layer = BatchNormalization()(layer)
        layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
        layer = BatchNormalization()(layer)
        layer = Dropout(0.5)(layer)
        layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
        layer = Dropout(0.5)(layer)
        layer = Flatten()(layer)
        layer = Dense(300, activation='sigmoid')(layer)  # 300
        layer = Dropout(0.5)(layer)
        layer = Dense(200, activation='sigmoid')(layer)  # 300
        predictions = Dense(7, activation='linear')(layer)
        norm_predictions = CustomNormalization()(predictions)  # Assuming "predictions" is your existing output

        model = Model(inputs=inputs, outputs=norm_predictions)
        model.summary()

        # compile model
        model.compile(optimizer= Adam(clipvalue=1.0),  # You can adjust the clipvalue as needed
                    loss=MaskedMSE(mask_value=-1),
                    metrics=["mse"]
                    )

        # Run model
        history=model.fit(data_reader(input_path_train, batch_size=100),
                                epochs=20,
                                validation_data=data_reader(input_path_valid,batch_size=500),
                                callbacks=None,
                                verbose=1)    

        # We predict the test data
        predicted = model.predict(data_reader(input_path_test, batch_size=500))

        # Sum the predicted values (will be averaged at the end of the 10 iterations)
        predictions_sum =  np.add(predictions_sum, predicted)
        
        
        # We reed the data in the same order to compute the correlation score
        test_data = data_reader(input_path_test,batch_size=500)
        test_tensor = np.empty(shape=[0,7])
        for batch in test_data:
            test_tensor = np.append(test_tensor, batch[1], axis=0)
            
        # Append correlation coefficient and append to previous
        corr_coefficient = pearson_correlation_multi_head(test_tensor, predicted, mask_value=-1)
        correlation_ensemble.append(corr_coefficient)
        avg_corr_coefficient = 1
        
        
    # We fill the dataframe with predictions and fold annotation
    
    predicted = predictions_sum/10 # We divide by the number of iterations
    
    df_test["avg_prediction_State_1M"] = predicted[:,0]
    df_test["avg_prediction_State_2D"] = predicted[:,1]
    df_test["avg_prediction_State_3E"] = predicted[:,2]
    df_test["avg_prediction_State_4M"] = predicted[:,3]
    df_test["avg_prediction_State_5M"] = predicted[:,4]
    df_test["avg_prediction_State_6N"] = predicted[:,5]
    df_test["avg_prediction_State_7M"] = predicted[:,6]
    df_test["fold"] = str(i)
    df_test["partition"] = "test"
    corr_coefficients["fold"+str(i)] = avg_corr_coefficient

    
    # Append fold to previous folds
    df_test_10folds = pd.concat([df_test_10folds, df_test], ignore_index=True)    
    

# Save the results for all folds
df_test_10folds.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/ensemble_multihead_motifs_2bp_extended_LibA_wide_pivot_state3_test_predicted_cv10fold.csv", index=False)

model_path = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/motifs_2bp_added_test_multihead_last_cv.h5"
model.save_weights(model_path, save_format='h5') 

df_test_10folds

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 262, 4)]          0         
                                                                 
 conv1 (Conv1D)              (None, 256, 250)          7250      
                                                                 
 dropout (Dropout)           (None, 256, 250)          0         
                                                                 
 batch_normalization (Batch  (None, 256, 250)          1000      
 Normalization)                                                  
                                                                 
 conv2 (Conv1D)              (None, 249, 250)          500250    
                                                                 
 batch_normalization_1 (Bat  (None, 249, 250)          1000      
 chNormalization)                                            

2023-11-28 13:55:23.567112: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-28 13:55:23.615422: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-28 13:55:23.733706: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-28 13:55:25.964792: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fc9dcf70b50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-28 13:55:25.964825: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-11-28 13:55:25.977626: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-28 13:55:26.067578: I ./tensorflow/compiler/jit/device_comp

     77/Unknown - 8s 24ms/step - loss: 0.0655 - mse: 0.3630

2023-11-28 13:55:30.166533: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 12493331387225790938
2023-11-28 13:55:30.166605: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 12757367510462626376
2023-11-28 13:55:30.166624: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16908915039380268482


77/77 [==============================] - 10s 41ms/step - loss: 0.0655 - mse: 0.3630 - val_loss: 0.0492 - val_mse: 0.4431
Epoch 2/20
77/77 [==============================] - 2s 19ms/step - loss: 0.0277 - mse: 0.3189 - val_loss: 0.0466 - val_mse: 0.4397
Epoch 3/20
77/77 [==============================] - 2s 20ms/step - loss: 0.0225 - mse: 0.3120 - val_loss: 0.0334 - val_mse: 0.3749
Epoch 4/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0193 - mse: 0.3085 - val_loss: 0.0258 - val_mse: 0.3325
Epoch 5/20
77/77 [==============================] - 2s 23ms/step - loss: 0.0174 - mse: 0.3075 - val_loss: 0.0257 - val_mse: 0.3218
Epoch 6/20
77/77 [==============================] - 2s 23ms/step - loss: 0.0162 - mse: 0.3056 - val_loss: 0.0237 - val_mse: 0.3069
Epoch 7/20
77/77 [==============================] - 2s 23ms/step - loss: 0.0151 - mse: 0.3055 - val_loss: 0.0224 - val_mse: 0.3009
Epoch 8/20
77/77 [==============================] - 2s 23ms/step - loss: 0.0151 - mse: 0.3044

...

2023-11-28 15:00:05.612230: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17334157443349932638


77/77 [==============================] - 2s 21ms/step - loss: 0.0231 - mse: 0.3103 - val_loss: 0.0501 - val_mse: 0.2520
Epoch 3/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0187 - mse: 0.3052 - val_loss: 0.0372 - val_mse: 0.2553
Epoch 4/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0170 - mse: 0.3032 - val_loss: 0.0350 - val_mse: 0.2625
Epoch 5/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0157 - mse: 0.3019 - val_loss: 0.0358 - val_mse: 0.2559
Epoch 6/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0153 - mse: 0.3015 - val_loss: 0.0314 - val_mse: 0.2723
Epoch 7/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0147 - mse: 0.3011 - val_loss: 0.0274 - val_mse: 0.2767
Epoch 8/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0140 - mse: 0.3000 - val_loss: 0.0203 - val_mse: 0.2777
Epoch 9/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0141 - mse: 0.2998 

2023-11-28 15:00:44.138344: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17334157443349932638
2023-11-28 15:00:44.138418: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 12757367510462626376


77/77 [==============================] - 2s 21ms/step - loss: 0.0222 - mse: 0.3095 - val_loss: 0.0470 - val_mse: 0.3183
Epoch 3/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0171 - mse: 0.3041 - val_loss: 0.0426 - val_mse: 0.3017
Epoch 4/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0157 - mse: 0.3024 - val_loss: 0.0349 - val_mse: 0.3059
Epoch 5/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0149 - mse: 0.3006 - val_loss: 0.0321 - val_mse: 0.3073
Epoch 6/20
77/77 [==============================] - 2s 21ms/step - loss: 0.0141 - mse: 0.3008 - val_loss: 0.0303 - val_mse: 0.2919
Epoch 7/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0140 - mse: 0.2998 - val_loss: 0.0277 - val_mse: 0.2828
Epoch 8/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0136 - mse: 0.2995 - val_loss: 0.0184 - val_mse: 0.2863
Epoch 9/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0135 - mse: 0.2996 

,State_3E,seq,avg_prediction_State_1M,avg_prediction_State_2D,avg_prediction_State_3E,avg_prediction_State_4M,avg_prediction_State_5M,avg_prediction_State_6N,avg_prediction_State_7M,CRS,State_1M,State_2D,State_4M,State_5M,State_6N,State_7M,fold,partition
0,-0.003656,aggaccggatcaactaaacaactcaaacaagggctaatataaccca...,0.095570,0.105198,0.076451,0.051704,0.070099,0.085040,0.080504,LibA.Seq7829,0.220825,0.091926,0.069263,NaN,-0.184409,NaN,10,test
1,0.163263,aggaccggatcaactaaacactagtcatacttaaaaattgcaagga...,0.048650,0.054372,0.031701,0.016734,0.032336,0.022117,0.053463,LibA.Seq271,0.113779,0.063990,0.075333,NaN,0.060112,NaN,10,test
2,-0.059611,aggaccggatcaactaaacaggttctgacgtatgctcctctatgga...,0.054560,0.058487,0.033275,0.020224,0.037792,0.030084,0.055674,LibA.Seq4548,-0.091269,0.008748,-0.032317,NaN,-0.123168,NaN,10,test
3,-0.060434,aggaccggatcaactaaacccgagcctgcctagccctagcttctct...,0.032921,0.042277,0.022726,0.013786,0.023908,0.011812,0.048435,LibA.Seq4582,0.011752,-0.038761,0.014523,0.051586,-0.036038,0.068024,10,test
4,0.255777,aggaccggatcaactaaacggagcagagttagtgtcaggtcaaaaa...,0.115457,0.121782,0.098389,0.059920,0.086326,0.103815,0.091696,LibA.Seq2863,0.269027,0.181167,0.099999,NaN,0.175921,NaN,10,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8487,-0.052776,aggaccggatcaacttttggtcggttgacggtcgccttgattattc...,0.039143,0.038899,0.026151,0.002910,0.027261,0.034528,0.033933,LibA.Seq4154,-0.012530,-0.020680,-0.033009,-0.013301,-0.118172,-0.027401,10,test
8488,0.254768,aggaccggatcaactttttcagtgaaagatcaccgcgggatctcac...,0.227602,0.214666,0.183796,0.128512,0.177952,0.264187,0.154990,LibA.Seq8531,0.320753,0.342002,0.172941,0.249537,0.382781,0.211203,10,test
8489,-0.067905,aggaccggatcaacttttttagtaaaactcttaaacagtgattaca...,0.205519,0.169651,0.187077,0.107923,0.168786,0.231961,0.160151,LibA.Seq6744,0.073526,0.130104,0.203765,NaN,NaN,NaN,10,test
8490,0.113337,aggaccggatcaacttttttatctggttatcattctagtctagtgc...,0.056722,0.054331,0.039518,0.011028,0.039420,0.051271,0.041265,LibA.Seq1298,0.090932,0.186463,0.107309,NaN,0.218181,0.209876,10,test


In [9]:
corr_coefficients

{'fold10': 1}

In [ ]:
df_test_10folds
